# 시멘틱 검색과 RAG


## 시멘틱 검색과 RAG 소개


- 밀집검색
  - 밀집 검색 시스템은 임베딩 개념을 사용하여 검색 문제를 검색 쿼리의 최근접 이웃을 찾는 것으로 변환합니다.
  - 텍스트 임베딩의 유사도를 기반으로 관련된 결과를 추출합니다.
- 리랭킹
  - 검색 시스템은 여러 단계로 구성된 파이프라인일 경우가 많습니다. 이런 단계 중 하나이며 쿼리와 결과의 관련성을 점수화합니다.
  - 검색 쿼리와 검색 결과를 받아 관련성에 따라 순위를 조정합니다.
- RAG
  - 검색 기능을 통합하여 환각을 줄이며 사실성을 높이고 생성모델을 특정 데이터셋에 접목한 텍스트 생성 시스템입니다.


## 언어 모델을 사용한 시멘틱 검색

### 밀집검색

- 임베딩은 텍스트를 수치 표현으로 바꾸며, 어떤 공간 위에 놓인 한 점으로 생각할수 있습니다.
- 비슷한 의미의 텍스트는 서로 가까이 놓입니다.
- 사용자가 입력한 검색 쿼리를 임베딩하고 텍스트 아카이브와 동일한 공간(차원이 동일한 벡터를 만든다는 의미)에 투영합니다.
- 그다음 이 공간상에서 쿼리에 가장 가까운 문서를 찾습니다.
- 나온 거리에서 약간 멀리 떨어진 텍스트도 반환해야 할까요?
  - 이 결정은 시스템 설계자에게 달려 있습니다.
  - 유사도 점수에 임곗값을 설정해 관련 없는 결과를 제외하는 것이 낫습니다.
- 쿼리와 가장 가까운 텍스트가 의미적으로 비슷한가요?
  - 항상 그렇지는 않습니다. 이 때문에 언어 모델을 질문-답변 쌍에서 훈련하여 검색 성능을 높여야 합니다.
- 외부 지식 데이터를 벡터 데이터베이스로 변환합니다. 그 다음 이 벡터 데이터베이스에 쿼리하여 지식정보를 검색합니다.


- 밀집 검색 예제
- 코히어를 사용해 위키백과에 있는 인터스텔라 영화 페이지에 담긴 내용을 검색하는 밀집 검색 예제를 만들어 보겠습니다.
  - 간단한 처리과정을 통해 검색 대상 텍스트를 문장으로 나눕니다.
  - 문장을 임베딩합니다.
  - 검색 인덱스를 구축합니다.
  - 검색을 수행하고 결과를 확인합니다.


- 코히어(https://cohere.com)에 가입하고 발급받은 API 키를 다음 코드에 붙여 넣어야 합니다.
- 코히어는 시험용 키를 일정 한도 내에서 무료로 제공합니다.
- 먼저 필요한 라이브러리를 임포트합니다.


In [ ]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

# 코히어 API 키 설정
api_key = ""

In [11]:
# 코히어 클라이언트를 만듭니다.
co = cohere.ClientV2(api_key)

- 텍스트 문서 분할하기


In [ ]:
text = """
인터스텔라는 크리스토퍼 놀란이 감독하고 그의 형제인 조너선 놀란과 공동으로 각본을 쓴 2014년의 대작 SF 영화입니다. 
매튜 맥코너히, 앤 해서웨이, 제시카 채스테인, 빌 어윈, 엘렌 버스틴, 마이클 케인 등 앙상블 캐스트가 출연합니다. 
지구가 재앙적인 병충해와 기근으로 고통받는 디스토피아적 미래를 배경으로, 인류를 위한 새로운 보금자리를 찾아 우주를 여행하는 우주 비행사들의 이야기를 담고 있습니다.

이 영화의 시나리오는 조너선이 2007년에 개발한 각본에서 시작되었으며, 원래 스티븐 스필버그가 감독을 맡을 예정이었습니다. 
이론 물리학자 킵 손은 이 영화의 총괄 프로듀서이자 과학 자문으로 참여했으며, 관련 서적인 《인터스텔라의 과학》을 집필했습니다. 
이 영화는 린다 옵스트가 사망하기 전 프로듀서로서 참여한 마지막 작품입니다. 
촬영 감독 호이트 반 호이테마는 ​​파나비전 아나모픽 포맷의 35mm 필름과 IMAX 70mm 필름으로 촬영했습니다. 
촬영은 2013년 말에 시작되어 앨버타, 클라우스터, 로스앤젤레스에서 진행되었습니다. 
《인터스텔라》는 광범위한 특수 효과와 미니어처 효과를 사용했으며, DNEG 사가 추가적인 시각 효과를 제작했습니다.

영화 인터스텔라는 2014년 10월 26일 TCL 차이니즈 극장에서 시사회를 가진 후, 11월 5일 미국에서, 11월 7일 영국에서 극장 개봉했습니다. 
미국 배급은 파라마운트 픽처스가, 해외 배급은 워너 브라더스 픽처스가 맡았습니다. 
미국에서는 처음에는 필름으로 개봉되었고, 이후 디지털 영사기를 사용하는 극장으로 확대 상영되었습니다. 
이 영화는 상업적으로 큰 성공을 거두어 개봉 첫 주에 전 세계적으로 6억 8100만 달러의 수익을 올렸고, 이후 재개봉을 통해 7억 7380만 달러를 추가로 벌어들이며 2014년 최고 흥행작 10위에 올랐습니다. 
평론가들로부터 대체적으로 호평을 받았습니다. 
인터스텔라는 여러 상을 수상했는데, 특히 제 87회 아카데미 시상식에서 5개 부문 후보에 올라 시각 효과상을 수상했습니다.
"""

# 문장을 나누어 리스트로 만듭니다.
texts = text.split(".")

# 공백과 줄바꿈 문자를 삭제합니다.
texts = [t.strip(" \n") for t in texts]
texts

['인터스텔라는 크리스토퍼 놀란이 감독하고 그의 형제인 조너선 놀란과 공동으로 각본을 쓴 2014년의 대작 SF 영화입니다',
 '매튜 맥코너히, 앤 해서웨이, 제시카 채스테인, 빌 어윈, 엘렌 버스틴, 마이클 케인 등 앙상블 캐스트가 출연합니다',
 '지구가 재앙적인 병충해와 기근으로 고통받는 디스토피아적 미래를 배경으로, 인류를 위한 새로운 보금자리를 찾아 우주를 여행하는 우주 비행사들의 이야기를 담고 있습니다',
 '이 영화의 시나리오는 조너선이 2007년에 개발한 각본에서 시작되었으며, 원래 스티븐 스필버그가 감독을 맡을 예정이었습니다',
 '이론 물리학자 킵 손은 이 영화의 총괄 프로듀서이자 과학 자문으로 참여했으며, 관련 서적인 《인터스텔라의 과학》을 집필했습니다',
 '이 영화는 린다 옵스트가 사망하기 전 프로듀서로서 참여한 마지막 작품입니다',
 '촬영 감독 호이트 반 호이테마는 \u200b\u200b파나비전 아나모픽 포맷의 35mm 필름과 IMAX 70mm 필름으로 촬영했습니다',
 '촬영은 2013년 말에 시작되어 앨버타, 클라우스터, 로스앤젤레스에서 진행되었습니다',
 '《인터스텔라》는 광범위한 특수 효과와 미니어처 효과를 사용했으며, DNEG 사가 추가적인 시각 효과를 제작했습니다',
 '영화 인터스텔라는 2014년 10월 26일 TCL 차이니즈 극장에서 시사회를 가진 후, 11월 5일 미국에서, 11월 7일 영국에서 극장 개봉했습니다',
 '미국 배급은 파라마운트 픽처스가, 해외 배급은 워너 브라더스 픽처스가 맡았습니다',
 '미국에서는 처음에는 필름으로 개봉되었고, 이후 디지털 영사기를 사용하는 극장으로 확대 상영되었습니다',
 '이 영화는 상업적으로 큰 성공을 거두어 개봉 첫 주에 전 세계적으로 6억 8100만 달러의 수익을 올렸고, 이후 재개봉을 통해 7억 7380만 달러를 추가로 벌어들이며 2014년 최고 흥행작 10위에 올랐습니다',
 '평론가들로부터 대체적으로 호평을 받았습니다',
 '인터스텔라는 여러 상을 수상했는데, 특히 

- 문장 임베딩하기
- 문장 리스트를 코히어 API에 전송하여 각 텍스트에 대한 벡터를 얻습니다.
- 코히어 클라이언트의 embed 메서드는 최적의 임베딩을 위해 input_type 매개변수에 입력의 종류를 지정해야 합니다.
  - 시멘틱 검색의 경우 쿼리는 "search_query"로, 검색 대상 텍스트는 "search_document"로 지정해야 합니다.
  - 분류나 클러스터링의 경우 각각 "classification"과 "clustering"으로 지정해야 합니다.
  - 이미지를 임베딩하는 경우 "image"로 지정해야 합니다.
  - embed 메서드가 반환하는 객체의 embeddings 속성은 texts 매개변수에 전달한 텍스트에 대한 임베딩을 저장한 리스트입니다.
- embed 메서드의 model 매개변수에 임베딩에 사용할 모델을 지정할 수 있습니다.
  - 사용 가능한 전체 모델은 코히어 온라인 문서를 참고하세요
  - https://docs.cohere.com/reference/embed


In [16]:
# 임베딩을 만듭니다.
response = co.embed(
    texts=texts,
    model="embed-v4.0",
    input_type="search_document",
    embedding_types=["float"],
).embeddings

embeds = np.array(response.float_)
print(embeds.shape)

(16, 1536)


- 크기가 1536인 벡터가 16개 있습니다.


- 검색하기 전 검색 인덱스를 구축해야 합니다.
- 검색 인덱스는 임베딩을 저장하며 많은 데이터 포인트에서도 빠르게 최근접 이웃을 검색할 수 있도록 최적화되었습니다.
- FAISS는 페이스북에서 만든 벡터 유사도 기반 검색 라이브러리입니다.
  - https://github.com/facebookresearch/faiss
  - IndexFlatL2 클래스는 유클리드 거리 기반으로 최근접 이웃을 찾는 기본적인 인덱스를 생성합니다.
  - 벡터를 추가하는 add() 메서드와 벡터를 검색하는 search() 메서드는 모두 32비트 부동소수점 실수를 기대합니다.


In [17]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

- 인덱스 검색하기


In [18]:
def search(query, number_of_results=3):
    # 쿼리 임베딩을 만듭니다.
    query_embed = co.embed(
        texts=[query],
        model="embed-v4.0",
        input_type="search_query",
        embedding_types=["float"],
    ).embeddings.float_[0]

    # 최근접 이웃을 추출합니다.
    distances, similar_item_ids = index.search(
        np.float32([query_embed]), number_of_results
    )

    # 데이터프레임을 사용해 출력을 준비합니다.
    texts_np = np.array(
        texts
    )  # 인덱싱을 쉽게 하기 위해 텍스트 리스트를 넘파이 배열로 변환합니다.
    results = pd.DataFrame(
        data={"텍스트": texts_np[similar_item_ids[0]], "거리": distances[0]}
    )

    # 결과를 출력하고 반환합니다.
    print(f"쿼리: '{query}'\n최근접 이웃:")
    return results

In [43]:
query = "영화를 어디서 볼 수 있나요?"
results = search(query)
results

쿼리: '영화를 어디서 볼 수 있나요?'
최근접 이웃:


,텍스트,거리
0,영화 인터스텔라는 2014년 10월 26일 TCL 차이니즈 극장에서 시사회를 가진 ...,1.025083
1,"미국에서는 처음에는 필름으로 개봉되었고, 이후 디지털 영사기를 사용하는 극장으로 확...",1.089149
2,이 영화는 상업적으로 큰 성공을 거두어 개봉 첫 주에 전 세계적으로 6억 8100만...,1.241750


- 키워드 검색 함수를 정의해 확인할 수 있습니다.
- 대표적인 어휘 검색 방법인 BM25 알고리즘을 사용하겠습니다.
- 코히어 가이드 노트북 : https://github.com/cohere-ai/cohere-developer-experience/blob/main/notebooks/guides/rerank-demo.ipynb


In [32]:
# 한글 BM25 알고리즘을 사용한 키워드 검색 예제

from kiwipiepy import Kiwi
from rank_bm25 import BM25Plus  # Okapi 대신 Plus 사용 (문서양이 적을 경우 사용)

kiwi = Kiwi()


# 2. 형태소 분석 함수 정의 (명사, 동사, 형용사 등 의미 있는 품사만 추출)
def tokenize_korean(text):
    # Kiwi를 통해 형태소 분석
    tokens = kiwi.tokenize(text)
    # 일반명사(NNG), 고유명사(NNP), 동사(VV), 형용사(VA) 등 주요 키워드만 필터링
    return [t.form for t in tokens if t.tag in ["NNG", "NNP", "VV", "VA"]]


corpus = [
    "맛있는 사과가 나무에 열려 있습니다.",
    "기차를 타고 서울에서 부산까지 여행을 갑니다.",
    "사과 나무 아래에서 기차 소리를 듣습니다.",
    "오늘 점심은 맛있는 비빔밥을 먹었습니다.",
]

tokenized_corpus = [tokenize_korean(doc) for doc in corpus]

# BM25Plus 모델 사용
bm25 = BM25Plus(tokenized_corpus)

query = "맛있는 사과 여행"
tokenized_query = tokenize_korean(query)

doc_scores = bm25.get_scores(tokenized_query)

print(f"질의어 토큰: {tokenized_query}\n")
for i, score in enumerate(doc_scores):
    print(f"문서 {i + 1}: {corpus[i]}")
    print(f"BM25 점수: {score:.4f}")
    print("-" * 30)

질의어 토큰: ['맛있', '사과', '여행']

문서 1: 맛있는 사과가 나무에 열려 있습니다.
BM25 점수: 5.4558
------------------------------
문서 2: 기차를 타고 서울에서 부산까지 여행을 갑니다.
BM25 점수: 4.9186
------------------------------
문서 3: 사과 나무 아래에서 기차 소리를 듣습니다.
BM25 점수: 4.3583
------------------------------
문서 4: 오늘 점심은 맛있는 비빔밥을 먹었습니다.
BM25 점수: 4.3583
------------------------------


Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


In [31]:
# 문서양이 많을 때는 이것을 사용해도 됨

from kiwipiepy import Kiwi
from rank_bm25 import BM25Okapi

# 1. Kiwi 초기화
kiwi = Kiwi()


# 2. 형태소 분석 함수 정의 (명사, 동사, 형용사 등 의미 있는 품사만 추출)
def tokenize_korean(text):
    # Kiwi를 통해 형태소 분석
    tokens = kiwi.tokenize(text)
    # 일반명사(NNG), 고유명사(NNP), 동사(VV), 형용사(VA) 등 주요 키워드만 필터링
    return [t.form for t in tokens if t.tag in ["NNG", "NNP", "VV", "VA"]]


# 3. 샘플 데이터 (문서 리스트)
corpus = [
    "맛있는 사과가 나무에 열려 있습니다.",
    "기차를 타고 서울에서 부산까지 여행을 갑니다.",
    "사과 나무 아래에서 기차 소리를 듣습니다.",
    "오늘 점심은 맛있는 비빔밥을 먹었습니다.",
]

# 4. 문서 토큰화
tokenized_corpus = [tokenize_korean(doc) for doc in corpus]

# 5. BM25 모델 생성
bm25 = BM25Okapi(tokenized_corpus)

# 6. 질의어 설정 및 분석
query = "맛있는 사과 여행"
tokenized_query = tokenize_korean(query)

# 7. 점수 계산 및 결과 출력
doc_scores = bm25.get_scores(tokenized_query)

print(f"질의어: {query}")
print("-" * 30)

for i, score in enumerate(doc_scores):
    print(f"문서 {i + 1}: {corpus[i]}")
    print(f"BM25 점수: {score:.4f}")
    print("-" * 30)

# 가장 점수가 높은 문서 추출
top_n = bm25.get_top_n(tokenized_query, corpus, n=1)
print(f"최종 추천 문서: {top_n[0]}")

Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


질의어: 맛있는 사과 여행
------------------------------
문서 1: 맛있는 사과가 나무에 열려 있습니다.
BM25 점수: 0.0000
------------------------------
문서 2: 기차를 타고 서울에서 부산까지 여행을 갑니다.
BM25 점수: 0.7773
------------------------------
문서 3: 사과 나무 아래에서 기차 소리를 듣습니다.
BM25 점수: 0.0000
------------------------------
문서 4: 오늘 점심은 맛있는 비빔밥을 먹었습니다.
BM25 점수: 0.0000
------------------------------
최종 추천 문서: 기차를 타고 서울에서 부산까지 여행을 갑니다.


- BM25는 TF-IDF 알고리즘을 개선한 텍스트 랭킹 및 검색 알고리즘입니다.
- BM250kapi 클래스에 각 문서를 토큰으로 분할한 리스트를 전달한 후 get_scores() 메서드를 사용해 쿼리와의 유사도 점수를 얻습니다.
- https://en.wikipedia.org/wiki/Okapi_BM25
- https://github.com/dorianbrown/rank_bm25


In [ ]:
# 요건 영어 일때..
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string


def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)  # 구두점 제거
        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


def keyword_search(query, top_k=3, num_candidates=15):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{"corpus_id": idx, "score": bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x["score"], reverse=True)

    print("탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[:top_k]:
        print(
            "\t{:.3f}\t{}".format(
                hit["score"], texts[hit["corpus_id"]].replace("\n", " ")
            )
        )


In [33]:
tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(tokenize_korean(passage))

bm25 = BM25Okapi(tokenized_corpus)


def keyword_search(query, top_k=3, num_candidates=15):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(tokenize_korean(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{"corpus_id": idx, "score": bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x["score"], reverse=True)

    print("탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[:top_k]:
        print(
            "\t{:.3f}\t{}".format(
                hit["score"], texts[hit["corpus_id"]].replace("\n", " ")
            )
        )

100%|██████████| 16/16 [00:00<00:00, 1571.89it/s]


In [42]:
keyword_search("영화를 어디서 볼 수 있나요?")

입력 질문: 영화를 어디서 볼 수 있나요?
탑-3 어휘 검색 (BM25) 결과
	0.511	이 영화는 린다 옵스트가 사망하기 전 프로듀서로서 참여한 마지막 작품입니다
	0.468	인터스텔라는 크리스토퍼 놀란이 감독하고 그의 형제인 조너선 놀란과 공동으로 각본을 쓴 2014년의 대작 SF 영화입니다
	0.449	이 영화의 시나리오는 조너선이 2007년에 개발한 각본에서 시작되었으며, 원래 스티븐 스필버그가 감독을 맡을 예정이었습니다


- 쿼리에 있는 영화가 포함되었지만 질문에 대한 답이 아닙니다.
- 리랭커를 추가하여 검색 시스템을 어떻게 개선할 수 있는지 알아보겠습니다.
